In [30]:
import numpy as np
import cv2
import os
from smart.video import Video, Image
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
import tensorflow as tf
import random
from sklearn.preprocessing import LabelEncoder
import pandas as pd

In [31]:
df = pd.read_csv('big_df.csv')

In [32]:
df.head()

,frame0pixel0,frame0pixel1,frame0pixel2,frame0pixel3,frame0pixel4,frame0pixel5,frame0pixel6,frame0pixel7,frame0pixel8,frame0pixel9,...,frame14pixel391,frame14pixel392,frame14pixel393,frame14pixel394,frame14pixel395,frame14pixel396,frame14pixel397,frame14pixel398,frame14pixel399,type
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,backhand
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,backhand
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,backhand
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,backhand
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,backhand


In [33]:
df.describe()

,frame0pixel0,frame0pixel1,frame0pixel2,frame0pixel3,frame0pixel4,frame0pixel5,frame0pixel6,frame0pixel7,frame0pixel8,frame0pixel9,...,frame14pixel390,frame14pixel391,frame14pixel392,frame14pixel393,frame14pixel394,frame14pixel395,frame14pixel396,frame14pixel397,frame14pixel398,frame14pixel399
count,1953.0,1953.0,1953.0,1953.0,1953.0,1953.0,1953.0,1953.0,1953.0,1953.000000,...,1953.000000,1953.000000,1953.000000,1953.000000,1953.000000,1953.000000,1953.000000,1953.0,1953.0,1953.0
mean,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001024,...,0.113671,0.084485,0.041475,0.009729,0.003584,0.000512,0.000512,0.0,0.0,0.0
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.031993,...,0.317493,0.278186,0.199436,0.098178,0.059776,0.022628,0.022628,0.0,0.0,0.0
min,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
25%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
50%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
75%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
max,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,0.0,0.0


In [34]:
X = df.drop(['type'], axis=1)
y = df['type']
y

0       backhand
1       backhand
2       backhand
3       backhand
4       backhand
          ...   
1948       smash
1949       smash
1950       smash
1951       smash
1952       smash
Name: type, Length: 1953, dtype: object

In [35]:
encoder = LabelEncoder()
y = encoder.fit_transform(y)
y

array([ 0,  0,  0, ..., 11, 11, 11])

In [36]:
output_y = np.unique(y)
output_y

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [37]:
np.array(X.iloc[0][0:400])

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [57]:
list_vid = []
for i in range(0, df.shape[0]):
    frames = []
    for j in range(0, 6000, 400):
        frame = np.array(X.iloc[i][j:j+400]) * 255.0
        frame = frame.reshape(20, 20)
        frames.append(frame)
    frames = np.array(frames)
    vid = Video.read_video_from_frames(frames)   
    list_vid.append(vid) 
len(list_vid)

1953

In [59]:
list_vid[0].frames.shape

(15, 20, 20)

In [62]:
train_size = int(len(list_vid) * 0.8)
test_size = len(list_vid) - train_size

In [63]:
## Split video train, test
random.seed(42)
vids_train = random.sample(list_vid, k = train_size)
vids_test = [vid for vid in list_vid if vid not in vids_train]
y_train = [y[i] for i in range(len(y)) if list_vid[i] in vids_train]
y_test = [y[i] for i in range(len(y)) if list_vid[i] in vids_test]
len(vids_train), len(y_train), len(vids_test), len(y_test)

(1562, 1562, 391, 391)

In [64]:
y_train = np.array(y_train, dtype=int)
y_test = np.array(y_test, dtype=int)

In [65]:
# Split input X en train, test set
X_train = np.zeros((train_size, 20, 15*20, 1), dtype=int)
for i, vid in enumerate(vids_train):
    for j, frame in enumerate(vid.frames):
        X_train[i, :, 20*j:20*(j+1), :] = frame.reshape(20, 20, 1)
X_train.shape

(1562, 20, 300, 1)

In [66]:
X_test = np.zeros((test_size, 20, 15*20, 1), dtype=int)
for i, vid in enumerate(vids_test):
    for j, frame in enumerate(vid.frames):
        X_test[i, :, 20*j:20*(j+1), :] = frame.reshape(20, 20, 1)
X_test.shape

(391, 20, 300, 1)

In [74]:
tf.random.set_seed(42)
model_cnn = Sequential(
    [
        tf.keras.Input(shape = (20, 300, 1)), # 50, 15*50, 3
        Conv2D(filters=16, kernel_size=(3, 3), activation="relu"),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.2),
        
        # Conv2D(filters=32, kernel_size=(3, 3), activation="relu"),
        # BatchNormalization(),
        # MaxPooling2D(pool_size=(2, 2)),
        # Dropout(0.2),
        
        Conv2D(filters=64, kernel_size=(3, 3), activation="relu"),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.2),
        
        Flatten(),
        Dense(units=128, activation="relu"),
        Dense(units=len(output_y), activation="softmax"),
    ]
)

model_cnn.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_22 (Conv2D)          (None, 18, 298, 16)       160       
                                                                 
 batch_normalization_22 (Bat  (None, 18, 298, 16)      64        
 chNormalization)                                                
                                                                 
 max_pooling2d_22 (MaxPoolin  (None, 9, 149, 16)       0         
 g2D)                                                            
                                                                 
 dropout_22 (Dropout)        (None, 9, 149, 16)        0         
                                                                 
 conv2d_23 (Conv2D)          (None, 7, 147, 64)        9280      
                                                                 
 batch_normalization_23 (Bat  (None, 7, 147, 64)      

In [71]:
checkpoint_name = 'checkpoints/Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [72]:
model_cnn.compile(
    loss = tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer=Adam(learning_rate=0.001),
    metrics=['accuracy']
)

history_cnn = model_cnn.fit(
    X_train, y_train,
    epochs = 50,
    batch_size = 32, # données transmises pour une session
    validation_split=0.2,
    callbacks=callbacks_list
)

Epoch 1/50
39/40 [============================>.] - ETA: 0s - loss: 6.6647 - accuracy: 0.1106
Epoch 1: val_loss improved from inf to 2.57394, saving model to checkpoints\Weights-001--2.57394.hdf5
40/40 [==============================] - 2s 51ms/step - loss: 6.6614 - accuracy: 0.1105 - val_loss: 2.5739 - val_accuracy: 0.0000e+00
Epoch 2/50
39/40 [============================>.] - ETA: 0s - loss: 2.4767 - accuracy: 0.1010
Epoch 2: val_loss did not improve from 2.57394
40/40 [==============================] - 2s 45ms/step - loss: 2.4767 - accuracy: 0.1009 - val_loss: 2.6182 - val_accuracy: 0.0000e+00
Epoch 3/50
39/40 [============================>.] - ETA: 0s - loss: 2.4579 - accuracy: 0.1106
Epoch 3: val_loss did not improve from 2.57394
40/40 [==============================] - 2s 44ms/step - loss: 2.4579 - accuracy: 0.1105 - val_loss: 2.6956 - val_accuracy: 0.0000e+00
Epoch 4/50
38/40 [===========================>..] - ETA: 0s - loss: 2.4375 - accuracy: 0.1135
Epoch 4: val_loss did not 

In [73]:
model_cnn.evaluate(X_test, y_test)

13/13 [==============================] - 0s 21ms/step - loss: 11.7318 - accuracy: 0.0793


[11.731772422790527, 0.07928388565778732]